In [1]:
import numpy as np
import csv
import pandas as pd
import re
import nltk
import random


ModuleNotFoundError: No module named 'tensorflow'

In [10]:
# build Vocabulary
from nltk import word_tokenize

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam

In [4]:
project_path1 = 'C:/Users/poorn/CyberSecData/'
file_path1 = project_path1 + 'GL_Bot_New.json'

In [5]:
# Read the json corpus data
import json
json_data = json.loads( open( file_path1 ).read( ) )

In [6]:
print(json_data)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [83]:
corps_voc = set()
corps_tags = []
corps_word_tag = []

{"intents": [
        {"tag": "Intro",
         "patterns": ["hi", 
                      "how are you", 
                      "online",
                      "i am from",
                      "hey ya",
                      "talking to you for first time"],
         "responses": ["Hello! how can i help you ?"],
         "context_set": ""
        }

In [84]:
# Iterate JSOn data and load contents (Corpus has Intents that has a structure
# tag - High level group
# patterns - possible inputs from user for training the model
# response -  List of responses that can be provided randomly if more than one response is available
# 
for intent_itm in json_data['intents']:
    tag = intent_itm['tag']
    corps_tags.append( tag ) #  add tag value
    for pattern in intent_itm['patterns']:
        words = word_tokenize( pattern )   #  Break the pattern row words
        corps_voc.update( words )          #  collect unique word
        corps_word_tag.append( ( words, tag ) )   
        

In [85]:
corps_voc = list( corps_voc )  # change from set to list

In [86]:
from nltk.stem import WordNetLemmatizer

In [87]:
# use WordNet alongside the NLTK module to find the meanings of words, synonyms, antonyms, and more. 
nltk.download('wordnet') 
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\poorn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\poorn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [88]:
# Create a list of spl characters
spl_chars = ['~', ':', "'", '+', '[', '\\', '@', '^', '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '!', '>', ';', '?', '#', '$', ')', '/']
wnl = WordNetLemmatizer()
corps_voc = [ wnl.lemmatize(word)  for word in corps_voc if word not in spl_chars]

In [89]:
from nltk.corpus import wordnet


In [90]:
training_data = []
output_empty = [0] * len(corps_tags)
output_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [91]:
print(len(corps_word_tag)) # No of sentences in [patterns]
print(len(corps_voc))  #  # of words across all sentences in [patterns]
print(len(corps_tags))

139
184
9


In [92]:
for docu in corps_word_tag:
    print(docu[0], docu[1])
    

['hi'] Intro
['how', 'are', 'you'] Intro
['is', 'anyone', 'there'] Intro
['hello'] Intro
['whats', 'up'] Intro
['hey'] Intro
['yo'] Intro
['listen'] Intro
['please', 'help', 'me'] Intro
['i', 'am', 'learner', 'from'] Intro
['i', 'belong', 'to'] Intro
['aiml', 'batch'] Intro
['aifl', 'batch'] Intro
['i', 'am', 'from'] Intro
['my', 'pm', 'is'] Intro
['blended'] Intro
['online'] Intro
['i', 'am', 'from'] Intro
['hey', 'ya'] Intro
['talking', 'to', 'you', 'for', 'first', 'time'] Intro
['thank', 'you'] Exit
['thanks'] Exit
['cya'] Exit
['see', 'you'] Exit
['later'] Exit
['see', 'you', 'later'] Exit
['goodbye'] Exit
['i', 'am', 'leaving'] Exit
['have', 'a', 'Good', 'day'] Exit
['you', 'helped', 'me'] Exit
['thanks', 'a', 'lot'] Exit
['thanks', 'a', 'ton'] Exit
['you', 'are', 'the', 'best'] Exit
['great', 'help'] Exit
['too', 'good'] Exit
['you', 'are', 'a', 'good', 'learning', 'buddy'] Exit
['olympus'] Olympus
['explain', 'me', 'how', 'olympus', 'works'] Olympus
['I', 'am', 'not', 'able', 't

In [70]:
for document in corps_word_tag:
    bag = []
    word_patterns = document[0]
    #word_patterns = [ wnl.lemmatize (word.lower())  for word in word_patterns]
    word_patterns = [ wnl.lemmatize ( x.lower() )  for x in word_patterns ]
    for word in corps_voc:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)

    output_row[ corps_tags.index(document[1])] = 1
    print(output_row)
    training_data.append( [ bag,  output_row])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[

In [99]:
# Prepare training data 
training_data = []
output_empty = [0] * len(corps_tags)
for document in corps_word_tag:
  bag = []
  word_patterns = document[0]
  word_patterns = [wnl.lemmatize (word.lower()) for word in word_patterns]
  for word in corps_voc:
    bag.append(1) if word in word_patterns else bag. append (0)
  output_row = list (output_empty)
  output_row[corps_tags.index(document[1])] = 1
  training_data.append ([bag, output_row])

In [106]:
print(len(training_data))
for td in training_data:
    print(len(td[0]),  len(td[1]))

139
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9
184 9


In [109]:
#random.shuffle(training_data)
Data_type = str
training_data = np.array(training_data, dtype=object)
X_train = list(training_data[:, 0])
y_train = list(training_data[:, 1])

In [110]:
print(training_data.shape)
print(len(X_train[0]))
print(len(y_train[0]))

(139, 2)
184
9


In [111]:
# define the model
model = Sequential()

model.add (Dense (128, input_shape=(len(X_train[0]),), activation='relu'))
model.add (Dropout(0.5))
model.add( Dense( 64, activation='relu' ) )
model.add (Dropout(0.5))
model.add( Dense( 32, activation='relu' ) )

model.add( Dense( len(y_train[0]), activation='softmax') )

In [112]:
opt_sgd = SGD(learning_rate=0.01)
model.compile( loss='categorical_crossentropy', optimizer=opt_sgd , metrics=['accuracy'])

In [113]:
# train the model by fitting the training data
history=model.fit(np.array(X_train), np.array(y_train), epochs=200, verbose=1, batch_size=5)

Epoch 1/200
28/28 [==============================] - 0s 1ms/step - loss: 2.2223 - accuracy: 0.0863
Epoch 2/200
28/28 [==============================] - 0s 1ms/step - loss: 2.2023 - accuracy: 0.1367
Epoch 3/200
28/28 [==============================] - 0s 1ms/step - loss: 2.1801 - accuracy: 0.1511
Epoch 4/200
28/28 [==============================] - 0s 945us/step - loss: 2.1770 - accuracy: 0.1583
Epoch 5/200
28/28 [==============================] - 0s 963us/step - loss: 2.1620 - accuracy: 0.1439
Epoch 6/200
28/28 [==============================] - 0s 963us/step - loss: 2.1503 - accuracy: 0.2374
Epoch 7/200
28/28 [==============================] - 0s 908us/step - loss: 2.1379 - accuracy: 0.2302
Epoch 8/200
28/28 [==============================] - 0s 926us/step - loss: 2.1336 - accuracy: 0.2014
Epoch 9/200
28/28 [==============================] - 0s 926us/step - loss: 2.1197 - accuracy: 0.2158
Epoch 10/200
28/28 [==============================] - 0s 968us/step - loss: 2.0846 - accuracy: 0.

28/28 [==============================] - 0s 926us/step - loss: 1.1879 - accuracy: 0.6043
Epoch 82/200
28/28 [==============================] - 0s 963us/step - loss: 1.1752 - accuracy: 0.5899
Epoch 83/200
28/28 [==============================] - 0s 1ms/step - loss: 1.1226 - accuracy: 0.6331
Epoch 84/200
28/28 [==============================] - 0s 963us/step - loss: 1.0513 - accuracy: 0.6619
Epoch 85/200
28/28 [==============================] - 0s 979us/step - loss: 1.1413 - accuracy: 0.6547
Epoch 86/200
28/28 [==============================] - 0s 1ms/step - loss: 1.0867 - accuracy: 0.6835
Epoch 87/200
28/28 [==============================] - 0s 965us/step - loss: 1.0769 - accuracy: 0.6115
Epoch 88/200
28/28 [==============================] - 0s 889us/step - loss: 1.0669 - accuracy: 0.6331
Epoch 89/200
28/28 [==============================] - 0s 889us/step - loss: 1.0210 - accuracy: 0.6763
Epoch 90/200
28/28 [==============================] - 0s 1ms/step - loss: 1.0580 - accuracy: 0.6978

28/28 [==============================] - 0s 891us/step - loss: 0.4000 - accuracy: 0.9137
Epoch 162/200
28/28 [==============================] - 0s 889us/step - loss: 0.3517 - accuracy: 0.8705
Epoch 163/200
28/28 [==============================] - 0s 889us/step - loss: 0.3954 - accuracy: 0.9065
Epoch 164/200
28/28 [==============================] - 0s 926us/step - loss: 0.4179 - accuracy: 0.9065
Epoch 165/200
28/28 [==============================] - 0s 883us/step - loss: 0.3481 - accuracy: 0.8921
Epoch 166/200
28/28 [==============================] - 0s 852us/step - loss: 0.4024 - accuracy: 0.8777
Epoch 167/200
28/28 [==============================] - 0s 926us/step - loss: 0.3594 - accuracy: 0.8921
Epoch 168/200
28/28 [==============================] - 0s 945us/step - loss: 0.3217 - accuracy: 0.9137
Epoch 169/200
28/28 [==============================] - 0s 926us/step - loss: 0.3501 - accuracy: 0.9065
Epoch 170/200
28/28 [==============================] - 0s 889us/step - loss: 0.3265 - a

In [114]:
def clean_up_sentence (sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [wnl.lemmatize(word) for word in sentence_words]
  return sentence_words


def bag_of_words (sentence):
  sentence_words = clean_up_sentence (sentence)
  bag = [0] * len (corps_voc)
  for w in sentence_words:
    for i, word in enumerate(corps_voc) :
      if word == w:
        bag[i] = 1
  return np.array (bag)

In [115]:
def predict_response(sentence):
  bow = bag_of_words (sentence.lower())
  res = model.predict(np.array([bow]), verbose=0)[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list= []
  for r in results:
    return_list.append({'intent': corps_tags[r[0]], 'probability' : str(r[1])})
  return return_list

In [116]:
print(predict_response('AIML'))

[{'intent': 'GL', 'probability': '0.8038906'}]


In [117]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  result = 'I cannot understand,  please re-phrase the question'
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

print ("Bot Running")
print("Welcome , Hi I am the Chatbot ")

Bot Running
Welcome , Hi I am the Chatbot 


In [121]:
exit_intent = 'Exit'
chat_alive = True
while chat_alive:
  message = input("")
  ints = predict_response(message.lower())
  if(ints[0]['intent'] == exit_intent):
    chat_alive = False
  res = get_response (ints, json_data)
  print(res)

print('Good bye')  

i am from
Hello! how can i help you ?
online
Hello! how can i help you ?
not able to understand
Link: Machine Learning wiki 
good bye
I hope I was able to assist you, Good Bye
Good bye
